### Resources:
- [Data School](https://www.youtube.com/playlist?list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y)
- [Jupyter](http://nbviewer.jupyter.org/github/justmarkham/pandas-videos/blob/master/pandas.ipynb)
- [Chris Albon](http://chrisalbon.com/)
- [Modern Pandas (Part 1)](http://tomaugspurger.github.io/modern-1.html)

In [1]:
import pandas as pd

In [2]:
drinks = pd.read_csv('http://bit.ly/drinksbycountry')
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


In [3]:
# Drops a single column (axis=1 refers to columns)
drinks = drinks.drop('country', axis=1)
drinks.head()

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,0,0,0,0.0,Asia
1,89,132,54,4.9,Europe
2,25,0,14,0.7,Africa
3,245,138,312,12.4,Europe
4,217,57,45,5.9,Africa


In [4]:
# Drops a bunch (10) rows (axis=0 refers to rows)
drinks.drop(drinks.index[0:10], axis=0, inplace=True)
drinks.head()

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
10,21,46,5,1.3,Europe
11,122,176,51,6.3,North America
12,42,63,7,2.0,Asia
13,0,0,0,0.0,Asia
14,143,173,36,6.3,North America


### = Melt...
- [YouTube Explanation](https://youtu.be/otCriSKVV_8?t=1h41m6s)

In [5]:
tables = pd.read_html("http://www.basketball-reference.com/leagues/NBA_2016_games.html")
games = tables[0]

games.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,,.1,Notes
0,"Tue, Oct 27, 2015",8:00 pm,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,NaN
1,"Tue, Oct 27, 2015",8:00 pm,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,NaN
2,"Tue, Oct 27, 2015",10:30 pm,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,NaN
3,"Wed, Oct 28, 2015",7:30 pm,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,NaN
4,"Wed, Oct 28, 2015",7:30 pm,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,NaN


In [6]:
column_names = {'Date': 'date', 'Start (ET)': 'start',
                'Unamed: 2': 'box', 'Visitor/Neutral': 'away_team', 
                'PTS': 'away_points', 'Home/Neutral': 'home_team',
                'PTS.1': 'home_points', 'Unamed: 7': 'n_ot'}

games = (games.rename(columns=column_names)
    .dropna(thresh=4)
    [['date', 'away_team', 'away_points', 'home_team', 'home_points']]
    .assign(date=lambda x: pd.to_datetime(x['date'], format='%a, %b %d, %Y'))
    .set_index('date', append=True)
    .rename_axis(["game_id", "date"])
    .sort_index())
games.head()

,,away_team,away_points,home_team,home_points
game_id,date,,,,
0,2015-10-27,Detroit Pistons,106,Atlanta Hawks,94
1,2015-10-27,Cleveland Cavaliers,95,Chicago Bulls,97
2,2015-10-27,New Orleans Pelicans,95,Golden State Warriors,111
3,2015-10-28,Philadelphia 76ers,95,Boston Celtics,112
4,2015-10-28,Chicago Bulls,115,Brooklyn Nets,100


- How many days of rest did each team get between each game?

In [7]:
# Melt says, my observations are spread across two columsn (home and away team) but I have attached dates
tidy = pd.melt(games.reset_index(),
               id_vars=['game_id', 'date'], value_vars=['away_team', 'home_team'],
               value_name='team')
tidy.head()

,game_id,date,variable,team
0,0,2015-10-27,away_team,Detroit Pistons
1,1,2015-10-27,away_team,Cleveland Cavaliers
2,2,2015-10-27,away_team,New Orleans Pelicans
3,3,2015-10-28,away_team,Philadelphia 76ers
4,4,2015-10-28,away_team,Chicago Bulls


In [8]:
tidy['variable'].unique()

array(['away_team', 'home_team'], dtype=object)

### = Piviot Table...
- [Tutorial Blog](http://pbpython.com/pandas-pivot-table-explained.html)

In [11]:
df = pd.read_csv('https://raw.githubusercontent.com/dylanjorgensen/data/master/data/pivot.csv')
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status,Unnamed: 8
0,0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [13]:
# You can have multiple indexes as well. In fact, most of the pivot_table args can take multiple values via a list.
pd.pivot_table(df,index=["Name","Rep","Manager"])

Account      Name     Price  \
Manager       Rep                                                         
Craig Booker  Fritsch, Russel and Anderson      3.0  737550.0  1.000000   
              Trantow-Barrows                   1.0  714466.0  1.333333   
Daniel Hilton Kiehn-Spinka                      4.0  146832.0  2.000000   

                                             Status  
Manager       Rep                                    
Craig Booker  Fritsch, Russel and Anderson  35000.0  
              Trantow-Barrows               15000.0  
Daniel Hilton Kiehn-Spinka                  65000.0

In [15]:
# pd.pivot_table(df,index=["Manager","Rep"])
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"])

Price
Manager       Rep                                   
Craig Booker  Fritsch, Russel and Anderson  1.000000
              Trantow-Barrows               1.333333
Daniel Hilton Kiehn-Spinka                  2.000000

In [17]:
import numpy as np
# aggfunc can take a list of functions. Let’s try a mean using the numpy mean function and len to get a count.
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=[np.mean,len])

mean   len
                                               Price Price
Manager       Rep                                         
Craig Booker  Fritsch, Russel and Anderson  1.000000     1
              Trantow-Barrows               1.333333     3
Daniel Hilton Kiehn-Spinka                  2.000000     1

I think one of the confusing points with the pivot_table is the use of columns and values . Remember, columns are optional - they provide an additional way to segment the actual values you care about. The aggregation functions are applied to the values you list.

In [18]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],
               columns=["Product"],aggfunc=[np.sum])

sum
                                                  Price
Product                                    Debra Henley
Manager       Rep                                      
Craig Booker  Fritsch, Russel and Anderson            1
              Trantow-Barrows                         4
Daniel Hilton Kiehn-Spinka                            2

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html

In [19]:
# Unstack